# Census and GeoJSON Data EDA

In [1]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
import json

import itertools

from time import time
from datetime import datetime, timedelta

from shapely.geometry import Polygon

In [2]:
def optimize(df):
    '''
    Optimizes the data types in a pandas dataframe.
    '''
    dft = df.copy()
    # converts to datetime if possible
    dft = dft.apply(lambda col: pd.to_datetime(col, errors='ignore') if col.dtypes=='object' else col)
    # if there are less than half as many unique values as there are rows, convert to category
    for col in dft.select_dtypes(include='object'):
        if len(dft[col].unique()) / len(df[col]) < 0.5:
            dft[col] = dft[col].astype('category')
    # downcasts numeric columns if possible
    dft = dft.apply(lambda col: pd.to_numeric(col, downcast='integer') if col.dtypes=='int64' else col)
    dft = dft.apply(lambda col: pd.to_numeric(col, downcast='float') if col.dtypes=='float64' else col)
    return dft

# 1. import census data from `census.gov`

Demographic data taken from [census.gov](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/asrh/).

Since 2020 Census data have not been released yet, we will use 2019 population estimates.

Looking at the [data dictionary](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf), we will only save the data from `YEAR == 12`.

In [3]:
with urlopen('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/asrh/cc-est2019-alldata.csv') as response:
    dem_df = pd.read_csv(
        response,
        encoding='latin-1',        # to avoid unicode error
        dtype={'STATE':'str',
               'COUNTY':'str'}
    )
dem_df = dem_df.drop(columns='SUMLEV')         # SUMLEV == 50 for the 50 US states
dem_df = dem_df.loc[(dem_df['YEAR'] == 12) & (dem_df['AGEGRP'] == 0)]    # population estimate for 2019
dem_df = dem_df.drop(columns=['YEAR', 'AGEGRP'])
dem_df.head()

,STATE,COUNTY,STNAME,CTYNAME,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
209,01,001,Alabama,Autauga County,55869,27092,28777,20878,21729,5237,...,778,687,89,93,40,27,15,19,16,11
437,01,003,Alabama,Baldwin County,223234,108247,114987,94810,100388,9486,...,5144,4646,268,281,264,197,69,65,55,35
665,01,005,Alabama,Barbour County,24686,13064,11622,6389,5745,6311,...,509,408,63,50,61,26,1,0,14,8
893,01,007,Alabama,Bibb County,22394,11929,10465,8766,8425,2941,...,291,253,32,19,6,15,5,1,17,3
1121,01,009,Alabama,Blount County,57826,28472,29354,27258,28154,516,...,2794,2516,76,58,67,66,18,21,34,21


In [4]:
dem_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 209 to 716357
Data columns (total 77 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STATE         3142 non-null   object
 1   COUNTY        3142 non-null   object
 2   STNAME        3142 non-null   object
 3   CTYNAME       3142 non-null   object
 4   TOT_POP       3142 non-null   int64 
 5   TOT_MALE      3142 non-null   int64 
 6   TOT_FEMALE    3142 non-null   int64 
 7   WA_MALE       3142 non-null   int64 
 8   WA_FEMALE     3142 non-null   int64 
 9   BA_MALE       3142 non-null   int64 
 10  BA_FEMALE     3142 non-null   int64 
 11  IA_MALE       3142 non-null   int64 
 12  IA_FEMALE     3142 non-null   int64 
 13  AA_MALE       3142 non-null   int64 
 14  AA_FEMALE     3142 non-null   int64 
 15  NA_MALE       3142 non-null   int64 
 16  NA_FEMALE     3142 non-null   int64 
 17  TOM_MALE      3142 non-null   int64 
 18  TOM_FEMALE    3142 non-null   int64 
 19  WA

Notice that county names provided by the US census contain descriptive terms, such as 'County', whereas the NYTimes data does not.

In [5]:
# remove descriptive terms from county names, will use on other dataframes
def remove_county_terms(data, county_col):
    county_terms = ['County', 'Parish', 'Municipality']
    for term in county_terms:
        data[county_col] = data[county_col].str.replace(' ' + term, '')
    return data

In [6]:
# rename columns to better-match nytimes data (and personal preference)
dem_df.rename(
    columns={
        'STATE':'statefips',
        'COUNTY':'countyfips',
        'STNAME':'state',
        'CTYNAME':'county',
        'TOT_POP': 'total_pop'
    }, inplace=True
)
dem_df.columns = dem_df.columns.str.lower()

# create fips column
dem_df['fips'] = dem_df['statefips'] + dem_df['countyfips']
dem_df = dem_df.drop(columns=['statefips', 'countyfips'])

dem_df = remove_county_terms(dem_df, 'county')
    
dem_df.head()

,state,county,total_pop,tot_male,tot_female,wa_male,wa_female,ba_male,ba_female,ia_male,...,hwac_female,hbac_male,hbac_female,hiac_male,hiac_female,haac_male,haac_female,hnac_male,hnac_female,fips
209,Alabama,Autauga,55869,27092,28777,20878,21729,5237,6000,121,...,687,89,93,40,27,15,19,16,11,01001
437,Alabama,Baldwin,223234,108247,114987,94810,100388,9486,10107,903,...,4646,268,281,264,197,69,65,55,35,01003
665,Alabama,Barbour,24686,13064,11622,6389,5745,6311,5595,103,...,408,63,50,61,26,1,0,14,8,01005
893,Alabama,Bibb,22394,11929,10465,8766,8425,2941,1822,53,...,253,32,19,6,15,5,1,17,3,01007
1121,Alabama,Blount,57826,28472,29354,27258,28154,516,462,192,...,2516,76,58,67,66,18,21,34,21,01009


## check county names against NYTimes data

We eventually need to merge `nyt_df` and `pop_df`, so let's see how they match with each other:

In [7]:
with urlopen('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv') as response:
    nyt_df = pd.read_csv(
        response,
        dtype={'fips':'str'}
    )
nyt_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [8]:
county_diffs = list(set(nyt_df['county']) - set(dem_df['county']))
len(county_diffs)

84

In [9]:
sorted([str(f) for f in county_diffs])

['Adjuntas',
 'Aguada',
 'Aguadilla',
 'Aguas Buenas',
 'Aibonito',
 'Anasco',
 'Arecibo',
 'Arroyo',
 'Barceloneta',
 'Barranquitas',
 'Bayamon',
 'Cabo Rojo',
 'Caguas',
 'Camuy',
 'Canovanas',
 'Carolina',
 'Catano',
 'Cayey',
 'Ceiba',
 'Ciales',
 'Cidra',
 'Coamo',
 'Comerio',
 'Corozal',
 'Culebra',
 'Dorado',
 'Fajardo',
 'Florida',
 'Guanica',
 'Guayama',
 'Guayanilla',
 'Guaynabo',
 'Gurabo',
 'Hatillo',
 'Hormigueros',
 'Humacao',
 'Isabela',
 'Jayuya',
 'Joplin',
 'Juana Diaz',
 'Juncos',
 'Kansas City',
 'Lajas',
 'Lares',
 'Las Marias',
 'Las Piedras',
 'Loiza',
 'Luquillo',
 'Manati',
 'Maricao',
 'Maunabo',
 'Mayaguez',
 'Moca',
 'Morovis',
 'Naguabo',
 'Naranjito',
 'New York City',
 'Orocovis',
 'Patillas',
 'Penuelas',
 'Ponce',
 'Quebradillas',
 'Rincon',
 'Sabana Grande',
 'Saipan',
 'Salinas',
 'San German',
 'San Lorenzo',
 'San Sebastian',
 'Santa Isabel',
 'St. John',
 'St. Thomas',
 'Tinian',
 'Toa Alta',
 'Toa Baja',
 'Trujillo Alto',
 'Unknown',
 'Utuado',
 '

As expected, the census county data is missing all municipios from [Puerto Rico](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-puerto-rico-municipios.html) (`fips == 72`) as well as a couple from the Northern Mariana Islands (`fips == 69`), so we need to append that data to `pop_df`.

## make rows for New York City, Kansas City, and Joplin

### New York City

Since the NYTimes dataset treats `New York City`, `Kansas City`, and `Joplin` [as their own entities](https://github.com/nytimes/covid-19-data#geographic-exceptions), we need to add them to `pop_df`.

`New York City` is the combination of these five counties, [which are coterminous with the five boroughs](https://en.wikipedia.org/wiki/New_York_City#Boroughs):

- Bronx
- Kings
- New York
- Queens
- Richmond

In [10]:
boroughs = ['Bronx', 'Kings', 'New York', 'Queens', 'Richmond']

nyc_dem_df = dem_df[(dem_df['state'] == 'New York') & (dem_df['county'] == boroughs[0])].select_dtypes(include='number')

for b in boroughs[1:]:
    nyc_dem_df += dem_df[(dem_df['state'] == 'New York') & (dem_df['county'] == b)].select_dtypes(include='number').values

nyc_dem_df['state'] = 'New York'
nyc_dem_df['county'] = 'New York City'
nyc_dem_df['fips'] = 'nyc'
nyc_dem_df.head()

,total_pop,tot_male,tot_female,wa_male,wa_female,ba_male,ba_female,ia_male,ia_female,aa_male,...,hbac_female,hiac_male,hiac_female,haac_male,haac_female,hnac_male,hnac_female,state,county,fips
417449,8336817,3978439,4358378,2145238,2247804,1046937,1247761,57897,58600,597346,...,294492,65574,67225,21187,22500,8967,9468,New York,New York City,nyc


In [11]:
dem_df = dem_df.append(nyc_dem_df, ignore_index=True)
dem_df[dem_df['fips'] == 'nyc']

,state,county,total_pop,tot_male,tot_female,wa_male,wa_female,ba_male,ba_female,ia_male,...,hwac_female,hbac_male,hbac_female,hiac_male,hiac_female,haac_male,haac_female,hnac_male,hnac_female,fips
3142,New York,New York City,8336817,3978439,4358378,2145238,2247804,1046937,1247761,57897,...,922001,257362,294492,65574,67225,21187,22500,8967,9468,nyc


In [12]:
dem_df['white'] = (dem_df['nhwa_male']+dem_df['nhwa_female'])
dem_df['black'] = (dem_df['nhba_male']+dem_df['nhba_female'])
dem_df['asian'] = (dem_df['nhaa_male']+dem_df['nhaa_female'])
dem_df['hispanic'] = (dem_df['h_male']+dem_df['h_female'])

In [13]:
pop_cols = ['state', 'county', 'total_pop', 'fips', 'white', 'black', 'asian', 'hispanic']
pop_df = dem_df[pop_cols]
pop_df.tail()

,state,county,total_pop,fips,white,black,asian,hispanic
3138,Wyoming,Teton,23464,56039,19000,145,378,3554
3139,Wyoming,Uinta,20226,56041,17657,126,92,1871
3140,Wyoming,Washakie,7805,56043,6417,38,55,1108
3141,Wyoming,Weston,6927,56045,6236,45,113,285
3142,New York,New York City,8336817,nyc,2681976,1825848,1228598,2423590


### Kansas City and Joplin

Kansas City and Joplin both refer to cities that cross county borders in Missouri. Therefore, we have to get our information from [census.gov quickfacts](https://www.census.gov/quickfacts).

We'll use `'kc'`, and `'jm'` as our `fips` for these three cities.

In [14]:
pop_df2 = pd.DataFrame(
    [['Missouri',
      'Kansas City',
      495_327,
      'kc',
      int(0.601*495_327),
      int(0.290*495_327),
      int(0.027*495_327),
      int(0.102*495_327)],
     ['Missouri',
      'Joplin',
      50_925,
      'jm',
      int(0.876*50_925),
      int(0.032*50_925),
      int(0.019*50_925),
      int(0.048*50_925)
     ]]
    , columns=pop_cols)
pop_df2

,state,county,total_pop,fips,white,black,asian,hispanic
0,Missouri,Kansas City,495327,kc,297691,143644,13373,50523
1,Missouri,Joplin,50925,jm,44610,1629,967,2444


In [15]:
pop_df = pop_df.append(pop_df2, ignore_index=True)
pop_df.tail(5)

,state,county,total_pop,fips,white,black,asian,hispanic
3140,Wyoming,Washakie,7805,56043,6417,38,55,1108
3141,Wyoming,Weston,6927,56045,6236,45,113,285
3142,New York,New York City,8336817,nyc,2681976,1825848,1228598,2423590
3143,Missouri,Kansas City,495327,kc,297691,143644,13373,50523
3144,Missouri,Joplin,50925,jm,44610,1629,967,2444


# 2. import geojson for boundaries and census areas

In [16]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    county_json = json.load(response)

In [17]:
# inspect structure of geojson
county_json['features'][0]

{'type': 'Feature',
 'properties': {'GEO_ID': '0500000US01001',
  'STATE': '01',
  'COUNTY': '001',
  'NAME': 'Autauga',
  'LSAD': 'County',
  'CENSUSAREA': 594.436},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-86.496774, 32.344437],
    [-86.717897, 32.402814],
    [-86.814912, 32.340803],
    [-86.890581, 32.502974],
    [-86.917595, 32.664169],
    [-86.71339, 32.661732],
    [-86.714219, 32.705694],
    [-86.413116, 32.707386],
    [-86.411172, 32.409937],
    [-86.496774, 32.344437]]]},
 'id': '01001'}

## add areas to `county_json`

In [18]:
fips_to_add_to_json = list(set(nyt_df['fips']) - set([f['id'] for f in county_json['features']]))
fips_to_add_to_json

[nan, '78010', '02158', '69120', '78020', '78030', '46102', '69110']

The `plotly` county GeoJSON dataset is missing Kusilvak Census Area (`'02158'`) and Oglala Lakota County(`'46102'`), in addition to the three cities included in the NYTimes data (New York City, Kansas City, Joplin). 

**FUTURE WORK**: we will ignore the following entries in `nyt_df`:
- Northern Mariana Islands (`69xxx`)
- US Virgin Islands (`78xxx`)
- state totals from an unknown county source (`nan`)

GeoJSON data for these five areas compiled from [nomanatim](https://nominatim.openstreetmap.org/) and [polygons](http://polygons.openstreetmap.fr/):
- Search for the area at [nomanatim](https://nominatim.openstreetmap.org/).
- Select `details` from the relevant entry.
- Copy the numeric `code` under `OSM`, ignoring "relation". Eg. for New York City, copy `175905`.
- Search for the `code` at [polygons](http://polygons.openstreetmap.fr/).
- For our purposes, GeoJSONs were selected according to the following criteria: (1) sparsity of vertices (`NPoints`) and (2) accuracy of shape.

In [19]:
# new york city, ny
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/nyc.txt') as response:
    nyc_json = json.load(response)

# kansas city, mo
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/kcm.txt') as response:
    kcm_json = json.load(response)

# joplin, mo
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/jm.txt') as response:
    jm_json = json.load(response)

# oglala lakota county, nd
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/olsd.txt') as response:
    olsd_json = json.load(response)

# kusilvak census area, ak
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/kca.txt') as response:
    kca_json = json.load(response)

In [20]:
# https://stackoverflow.com/questions/41271146/
def clean_coordinates(c):
    return [list(itertools.chain(*d)) for d in c]

In [21]:
add_to_json_dict = {
    '02158':{'area':17_081.43,
             'name':'Kusilvak Census Area',
             'coordinates':clean_coordinates(kca_json['coordinates'])}, 
    '46102':{'area':2_093.90,
             'name':'Oglala Lakota',
             'coordinates':clean_coordinates(olsd_json['coordinates'])},
    'jm':{'area':35.56,
          'name':'Joplin',
          'coordinates':clean_coordinates(jm_json['geometries'][0]['coordinates'])},
    'kc':{'area':314.95,
          'name':'Kansas City',
          'coordinates':clean_coordinates(kcm_json['coordinates'])},
    'nyc':{'area':302.64,
           'name':'New York City',
           'coordinates':clean_coordinates(nyc_json['coordinates'])}
}

In [22]:
for fips in ['02158', '46102', 'jm', 'kc', 'nyc']:
    county_json['features'].append(
        {
            'geometry': {'coordinates': add_to_json_dict[fips]['coordinates'],
                         'type': 'Polygon'},
            'id': fips,
            'properties': {'NAME': add_to_json_dict[fips]['name'],
                           'CENSUSAREA': add_to_json_dict[fips]['area']},
            'type': 'Feature'
        }
    )

In [23]:
with open('data/county_json.json', 'w') as f:
    json.dump(county_json, f)

# 3. add centroid latitude and longitude coordinates and county area from `county_json` to `pop_df`

We will use `shapely` to calculate the [centroid](https://en.wikipedia.org/wiki/Centroid) coordinates for the counties (in case we wish to plot bubble maps).

In [79]:
x = np.array([list([[1,2], [1,2]])])
x

array([[[1, 2],
        [1, 2]]])

In [97]:
def centroid(i, j=county_json):
    for d in j['features']:
        if d['id'] == i:
            shapes = np.array(d['geometry']['coordinates'])
            # if county_json has multiple polygons
            if shapes.ndim != 2:
                areas = [Polygon(shape).area for shape in shapes]
                coords = [Polygon(shape).centroid.coords[0] for shape in shapes]
                lon = np.average(list(zip(*coords))[0], weights=areas)
                lat = np.average(list(zip(*coords))[1], weights=areas)
            # if county_json has one polygon
            else:
                shapes = np.reshape(shapes, (-1, 2))
                p = Polygon(shapes)
                lon, lat = p.centroid.coords[0]
            return lon, lat

In [93]:
def centroid2(i, j=county_json):
    for d in j['features']:
        if d['id'] == i:
            coords = np.array(d['geometry']['coordinates'])
            return coords

In [98]:
centroid('jm')

(-94.50566362499289, 37.07898462953337)

In [26]:
def county_area(i, j=county_json):
    for d in j['features']:
        if d['id'] == i:
            return d['properties']['CENSUSAREA']

In [27]:
tick = time()
pop_df['area'] = pop_df['fips'].apply(county_area)
pop_df['lon'], pop_df['lat'] = zip(*pop_df['fips'].apply(centroid).to_list())
# pop_df = optimize(pop_df)
tock = time()
print(tock - tick)

0.7041211128234863


In [28]:
pop_df.tail()

,state,county,total_pop,fips,white,black,asian,hispanic,area,lon,lat
3140,Wyoming,Washakie,7805,56043,6417,38,55,1108,2238.549,-107.681649,43.904771
3141,Wyoming,Weston,6927,56045,6236,45,113,285,2398.089,-104.567290,43.839661
3142,New York,New York City,8336817,nyc,2681976,1825848,1228598,2423590,302.640,-73.939368,40.663516
3143,Missouri,Kansas City,495327,kc,297691,143644,13373,50523,314.950,-94.554422,39.127195
3144,Missouri,Joplin,50925,jm,44610,1629,967,2444,35.560,-94.505664,37.078985


# 4. add 2016 general election data

Mask compliance has been very political, so it would be interesting to see how political differences vary by county. Data taken from [github.com/tonmcg](https://github.com/tonmcg). Alaska data taken from [RRH Elections](https://rrhelections.com/index.php/2018/02/02/alaska-results-by-county-equivalent-1960-2016/).

In [30]:
with urlopen('https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-16/master/2016_US_County_Level_Presidential_Results.csv') as response:
    elect_df = pd.read_csv(
        response,
        encoding='latin-1',        # to avoid unicode error
        dtype={
            'votes_dem':'int',
            'votes_gop':'int',
            'total_votes':'int',
            'combined_fips':'str'},
        index_col=0
    )
elect_df.head()

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060


In [31]:
elect_df.rename(
    columns={
        'county_name':'county',
        'combined_fips':'fips',
    }, inplace=True
)

# https://stackoverflow.com/a/23836353
elect_df['fips'] = elect_df['fips'].apply('{0:0>5}'.format)

elect_df = remove_county_terms(elect_df, 'county')

In [32]:
elect_df = elect_df[['state_abbr', 'county', 'fips', 'votes_dem', 'votes_gop', 'total_votes']]
elect_df = elect_df.sort_values(by='fips')
elect_df.head()

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
29,AL,Autauga,01001,5908,18110,24661
30,AL,Baldwin,01003,18409,72780,94090
31,AL,Barbour,01005,4848,5431,10390
32,AL,Bibb,01007,1874,6733,8748
33,AL,Blount,01009,2150,22808,25384


## add New York City, Kansas City, and Joplin election data

In [33]:
nyc_elect_df = elect_df[(elect_df['state_abbr'] == 'NY') & (elect_df['county'] == boroughs[0])].select_dtypes(include='number')

for b in boroughs[1:]:
    nyc_elect_df += elect_df[(elect_df['state_abbr'] == 'NY') & (elect_df['county'] == b)].select_dtypes(include='number').values
    
nyc_elect_df

,votes_dem,votes_gop,total_votes
1982,1969920,461174,2490750


In [34]:
# nyc_elect_df['per_dem'] = nyc_elect_df['votes_dem'] / nyc_elect_df['total_votes']
# nyc_elect_df['per_gop'] = nyc_elect_df['votes_gop'] / nyc_elect_df['total_votes']

In [35]:
nyc_elect_df['state_abbr'] = 'NY'
nyc_elect_df['county'] = 'New York City'
nyc_elect_df['fips'] = 'nyc'
nyc_elect_df.head()

,votes_dem,votes_gop,total_votes,state_abbr,county,fips
1982,1969920,461174,2490750,NY,New York City,nyc


In [36]:
elect_df = elect_df.append(nyc_elect_df, ignore_index=True)
elect_df.tail()

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
3137,WY,Teton,56039,7313,3920,12176
3138,WY,Uinta,56041,1202,6154,8053
3139,WY,Washakie,56043,532,2911,3715
3140,WY,Weston,56045,294,2898,3334
3141,NY,New York City,nyc,1969920,461174,2490750


## add alaska elections data

In [37]:
ak_elect_df = pd.read_excel('data/2016 AK Gen Official.xlsx', sheet_name='By CE')
ak_elect_df = ak_elect_df.iloc[0:29, 0:12]
ak_elect_df.head()

,ED/Muni,Municipality Code,Registered Voters,Times Counted,"Castle, Darrell L.","Clinton, Hillary","De La Fuente, Roque","Johnson, Gary","Stein, Jill","Trump, Donald J.",Write-in 60,ED Total
0,Ketchikan Gateway,Ketchikan,10512,4283,48,1295,13,339,84,2354,104,4237
1,Prince of Wales-Hyder,Prince of Wales-Hyder,4630,1831,67,666,29,93,65,831,59,1810
2,Sitka,Sitka,7218,2787,38,1261,18,145,78,1146,73,2759
3,Petersburg,Petersburg,2741,1078,12,334,7,64,37,577,32,1063
4,Wrangell,Wrangell,1731,764,7,177,3,35,13,512,13,760


In [38]:
ak_elect_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ED/Muni               29 non-null     object
 1   Municipality Code     29 non-null     object
 2   Registered Voters     29 non-null     object
 3   Times Counted         29 non-null     object
 4   Castle, Darrell L.    29 non-null     object
 5   Clinton, Hillary      29 non-null     object
 6   De La Fuente, Roque   29 non-null     object
 7   Johnson, Gary         29 non-null     object
 8   Stein, Jill           29 non-null     object
 9   Trump, Donald J.      29 non-null     object
 10  Write-in 60           29 non-null     object
 11  ED Total              29 non-null     object
dtypes: object(12)
memory usage: 2.8+ KB


In [39]:
ak_elect_df.rename(
    columns={
        'Trump, Donald J. ':'votes_gop',
        'Clinton, Hillary ':'votes_dem'
    }, inplace=True
)
ak_elect_df = ak_elect_df[['ED/Muni', 'votes_gop', 'votes_dem', 'ED Total']].sort_values(by='ED/Muni')
ak_elect_df[['votes_gop', 'votes_dem', 'ED Total']] = ak_elect_df[['votes_gop', 'votes_dem', 'ED Total']].astype(int)
ak_elect_df = ak_elect_df.sort_values(by='ED/Muni')
ak_elect_df.head()

,ED/Muni,votes_gop,votes_dem,ED Total
22,Aleutians East,198,121,369
24,Aleutians West,260,493,846
19,Anchorage,39942,32130,81678
12,Bethel,809,2178,3933
25,Bristol Bay,180,99,316


In [40]:
print(len(ak_elect_df))
print(len(elect_df[elect_df['state_abbr'] == 'AK']))

29
29


In [41]:
elect_df.loc[elect_df['state_abbr'] == 'AK', ['votes_gop', 'votes_dem', 'total_votes']] = ak_elect_df[['votes_gop', 'votes_dem', 'ED Total']].values

In [42]:
# elect_df = elect_df.drop(columns=['votes_dem', 'votes_gop'])
elect_df.tail()

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
3137,WY,Teton,56039,7313,3920,12176
3138,WY,Uinta,56041,1202,6154,8053
3139,WY,Washakie,56043,532,2911,3715
3140,WY,Weston,56045,294,2898,3334
3141,NY,New York City,nyc,1969920,461174,2490750


In [43]:
elect_df[elect_df['state_abbr'] == 'AK'].head()

,state_abbr,county,fips,votes_dem,votes_gop,total_votes
67,AK,Alaska,02013,121,198,369
68,AK,Alaska,02016,493,260,846
69,AK,Alaska,02020,32130,39942,81678
70,AK,Alaska,02050,2178,809,3933
71,AK,Alaska,02060,99,180,316


In [44]:
pop_df = pop_df.merge(elect_df[['fips', 'votes_gop', 'votes_dem', 'total_votes']], on='fips', how='left')

### adjusted 2-party voting

In [45]:
# pop_df['per_gop_adj'] = pop_df['per_gop'] / (pop_df['per_gop'] + pop_df['per_dem'])

# add income data

Median income statistics taken from [data.census.gov](https://data.census.gov/cedsci/table?q=s1901&tid=ACSST1Y2018.S1901) (2017 ACS 1-Year Estimates). According to the data table, the median household income is reported in column `S1901_C01_012E`.

In [46]:
income_df = pd.read_csv('data/ACSST5Y2018.S1901_data_with_overlays_2020-07-16T134009.csv',
                        usecols=['GEO_ID', 'NAME', 'S1901_C01_012E'])
income_df = income_df.drop(0, axis=0)
income_df.rename(
    columns={
        'GEO_ID':'fips',
        'S1901_C01_012E':'median_income'
    }, inplace=True
)
income_df['median_income'] = income_df['median_income'].astype(float)
income_df['fips'] = income_df['fips'].str[-5:]
income_df.head()

,fips,NAME,median_income
1,01001,"Autauga County, Alabama",58786.0
2,01003,"Baldwin County, Alabama",55962.0
3,01005,"Barbour County, Alabama",34186.0
4,01007,"Bibb County, Alabama",45340.0
5,01009,"Blount County, Alabama",48695.0


In [47]:
income_df[income_df['median_income'].isna()]

,fips,NAME,median_income
1817,35039,"Rio Arriba County, New Mexico",NaN


Rio Arriba Income statistics taken from [datausa.io](https://datausa.io/profile/geo/rio-arriba-county-nm#:~:text=Median%20household%20income%20in%20Rio%20Arriba%20County%2C%20NM%20is%20%2433%2C422.)

In [48]:
income_df.at[income_df['fips'] == '35039', 'median_income'] = 33_422

In [49]:
income_df['county'], income_df['state'] = zip(*income_df['NAME'].str.split(', ').tolist())
income_df = income_df.drop('NAME', axis=1)
income_df = remove_county_terms(income_df, 'county')
income_df['median_income'] = income_df['median_income'].astype(int)
income_df.head()

,fips,median_income,county,state
1,01001,58786,Autauga,Alabama
2,01003,55962,Baldwin,Alabama
3,01005,34186,Barbour,Alabama
4,01007,45340,Bibb,Alabama
5,01009,48695,Blount,Alabama


In [50]:
# only an estimate of median income

nyc_income = income_df[(income_df['state'] == 'New York') & (income_df['county'] == boroughs[0])]['median_income'].values \
                * pop_df[(pop_df['state'] == 'New York') & (pop_df['county'] == boroughs[0])]['total_pop'].values
nyc_pop = pop_df[(pop_df['state'] == 'New York') & (pop_df['county'] == boroughs[0])]['total_pop'].values

for b in boroughs[1:]:
    nyc_income += income_df[(income_df['state'] == 'New York') & (income_df['county'] == b)]['median_income'].values \
                     * pop_df[(pop_df['state'] == 'New York') & (pop_df['county'] == b)]['total_pop'].values
    nyc_pop += pop_df[(pop_df['state'] == 'New York') & (pop_df['county'] == b)]['total_pop'].values
    
nyc_income = nyc_income / nyc_pop
nyc_income[0]

61884.61918031786

In [51]:
income_df2 = pd.DataFrame(
    [['nyc', int(nyc_income[0]), 'New York City', 'New York'],
     ['kc', 52405, 'Kansas City', 'Missouri'],
     ['jm', 42782, 'Joplin', 'Missouri']]
    , columns=income_df.columns)
income_df2

,fips,median_income,county,state
0,nyc,61884,New York City,New York
1,kc,52405,Kansas City,Missouri
2,jm,42782,Joplin,Missouri


In [52]:
income_df = income_df.append(income_df2, ignore_index=True)
income_df.tail(5)

,fips,median_income,county,state
3218,72151,16013,Yabucoa Municipio,Puerto Rico
3219,72153,14954,Yauco Municipio,Puerto Rico
3220,nyc,61884,New York City,New York
3221,kc,52405,Kansas City,Missouri
3222,jm,42782,Joplin,Missouri


In [53]:
pop_df = pop_df.merge(income_df[['fips', 'median_income']], on='fips', how='left')
pop_df.head()

,state,county,total_pop,fips,white,black,asian,hispanic,area,lon,lat,pop_per_area,votes_gop,votes_dem,total_votes,median_income
0,Alabama,Autauga,55869,01001,41215,11098,646,1671,594.436,-86.641196,32.536153,93.986569,18110.0,5908.0,24661.0,58786
1,Alabama,Baldwin,223234,01003,185747,19215,2346,10534,1589.784,-87.723954,30.725862,140.417818,72780.0,18409.0,94090.0,55962
2,Alabama,Barbour,24686,01005,11235,11807,116,1117,884.876,-85.389245,31.867889,27.897694,5431.0,4848.0,10390.0,34186
3,Alabama,Bibb,22394,01007,16663,4719,46,623,622.582,-87.124963,32.996456,35.969559,6733.0,1874.0,8748.0,45340
4,Alabama,Blount,57826,01009,50176,872,163,5582,644.776,-86.569756,33.985248,89.683859,22808.0,2150.0,25384.0,48695


## save results to csv

In [54]:
dem_df.to_csv('data/dem_df.csv', index=False)
pop_df.to_csv('data/pop_df.csv', index=False)

# Future Work: import Puerto Rico census data

To do:
- find detailed demographic data for Puerto Rico
- find a way to incorporate Puerto Rico into the Altair map

In [ ]:
# with urlopen('https://www2.census.gov/programs-surveys/popest/tables/2010-2019/municipios/totals/prm-est2019-annres.xlsx') as response:
#     pr_df = pd.read_excel(response, header=3)
pr_df = pd.read_excel('data/prm-est2019-annres.xlsx', header=3)
pr_df = pr_df[['Unnamed: 0', 2019]]
pr_df.rename(
    columns={
        'Unnamed: 0':'county',
        2019:'total_pop'
    }, inplace=True
)
pr_df = pr_df[~pr_df['total_pop'].isna()]
pr_df['total_pop'] = pr_df['total_pop'].astype('int')
pr_df.head()

In [ ]:
pr_df['county'] = [s[0] if len(s) > 0 else s for s in pr_df['county'].str.findall("\.([\w\s]+) Municipio\,.+")]
pr_df = pr_df.iloc[1:]          # removing the territory as a whole from the table
pr_df.head()

We also need to add `fips` codes for all of the municipios.

### import Puerto Rico `fips`

In [ ]:
sess = HTMLSession()
res = sess.get('https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county')
table = res.html.find('table.wikitable > tbody > tr')
# puerto rico is fips 72
pr_fips = [[tr.find('td')[1].text, tr.find('td')[0].text] for tr in table[1:] if tr.find('td')[0].text[:2] == '72']
pr_fips_df = pd.DataFrame(pr_fips)
pr_fips_df.rename(
    columns={
        0:'county',
        1:'fips'
    }, inplace=True
)
pr_fips_df.head()

In [ ]:
pr_fips_df['county'] = [s[0] if len(s) > 0 else s for s in pr_fips_df['county'].str.findall("([\w\s]+) Municipality")]
pr_fips_df.head()

In [ ]:
len(list(set(pr_fips_df['county']) - set(pr_df['county'])))

In [ ]:
pr_df = pr_df.merge(pr_fips_df, on='county')
pr_df['state'] = 'Puerto Rico'
pr_df.head()

In [ ]:
pop_df = optimize(pop_df.append(pr_df, ignore_index=True).append(pr_df, ignore_index=True))
pop_df.tail()

## check county names against NYTimes data (again)

In [ ]:
county_diffs = list(set(nyt_df['county']) - set(dem_df['county']))
len(county_diffs)

In [ ]:
county_diffs

The NYTimes dataset is missing diacritical marks in their names. While it would be easier to replace diacritical marks with their "standard" character counterparts, we will preserve them in our final dataframe in the interest of cultural accuracy. This will be handled when we merge `pop_df` with `nyt_df` in the other notebook.